## Feature Engineering using wrapper method

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
shopper = pd.read_csv('online_shoppers_intention.csv', sep=',')
shopper.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [13]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder(dtype=int)
previous_shopper = shopper.copy()
shopper['Month', 'VisitorType', 'Weekend', 'Revenue'] = enc.fit_transform(shopper[['Month', 'VisitorType', 'Weekend', 'Revenue']])
shopper['VisitorType'] = enc.fit_transform(shopper[['VisitorType']])
shopper['Weekend'] = enc.fit_transform(shopper[['Weekend']])
shopper['Revenue'] = enc.fit_transform(shopper[['Revenue']])

In [15]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tqdm

rfe_f1_score_list = []
rfe_f1_score_dict = {}
X,y = shopper.drop(['Revenue'], axis=1), shopper['Revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42, shuffle=True)

for k in tqdm.tqdm(range(1,len(shopper.columns)+1)):
    estimator = SVC(kernel='linear')
    RFE_selector = RFE(estimator, step=1, n_features_to_select=k)
    RFE_selector = RFE_selector.fit(X, y)

    sel_X_train = RFE_selector.transform(X_train)
    sel_X_test = RFE_selector.transform(X_test)
    
    estimator.fit(sel_X_train, y_train)
    RFE_preds = estimator.predict(sel_X_test)
    
    f1_score_rfe = round(f1_score(y_test, RFE_preds),8)
    rfe_f1_score_dict[k] = RFE_selector.get_feature_names_out()
    rfe_f1_score_list.append(f1_score_rfe)
    
print(rfe_f1_score_list)

  0%|          | 0/18 [00:00<?, ?it/s]